# Домашнее задание №4



# Студент: Борцов Михаил Андреевич, РИМ-150950

## Задание
*Для изучения CSS селекторов https://flukeout.github.io/*

- Найдите сайт с открытыми таблицами (например, статистика IMDB, погодные данные, вакансии) и с помощью requests + BeautifulSoup извлеките несколько колонок и загрузите в DataFrame.

- Установите заголовок запроса (headers={'User-Agent': 'Mozilla/5.0 ...'}) и добавьте time.sleep(1) между запросами при парсинге нескольких страниц.

- Используйте soup.select() с CSS-селектором, чтобы выбрать интересующие элементы и извлечь текст или атрибуты.

- После получения данных с веба отметьте, как часто встречаются пропущенные значения или некорректные форматы (например, строка вместо числа)  и примените методы Pandas по очистке.

In [1]:
import requests, time, re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}

url = "https://www.imdb.com/chart/top/"

In [3]:
response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

## Находим список фильмов

In [4]:
film_list = soup.find('ul', class_="ipc-metadata-list")
rows = film_list.find_all('li', class_="ipc-metadata-list-summary-item")

## Собираем данные

In [5]:
movies = []
for row in rows:
    # Используем soup.select() с CSS-селекторами, как требуется в задании
    title_elem = row.select("h3.ipc-title__text")
    year_elem = row.select("span.sc-b4f120f6-7:nth-child(1)")
    rating_elem = row.select("span.ipc-rating-star--rating")
    duration_elem = row.select("span.sc-b4f120f6-7:nth-child(2)")

    # Извлекаем текст, если элемент существует
    title = title_elem[0].get_text(strip=True) if title_elem else None
    year = year_elem[0].get_text(strip=True) if year_elem else None
    rating = rating_elem[0].get_text(strip=True) if rating_elem else None
    duration = duration_elem[0].get_text(strip=True) if duration_elem else None

    movies.append({
        "name": title,
        "year": year,
        "rating": rating,
        "time": duration
    })

## Создаём DataFrame

In [6]:
df = pd.DataFrame(movies)
df

,name,year,rating,time
0,The Shawshank Redemption,1994,9.3,2h 22m
1,The Godfather,1972,9.2,2h 55m
2,The Dark Knight,2008,9.1,2h 32m
3,The Godfather Part II,1974,9.0,3h 22m
4,12 Angry Men,1957,9.0,1h 36m
5,The Lord of the Rings: The Return of the King,2003,9.0,3h 21m
6,Schindler's List,1993,9.0,3h 15m
7,The Lord of the Rings: The Fellowship of the Ring,2001,8.9,2h 58m
8,Pulp Fiction,1994,8.8,2h 34m
9,"The Good, the Bad and the Ugly",1966,8.8,2h 58m


## Проверка на пропущенные значения

In [7]:
print("Пропущенные значения:")
print(df.isnull().sum())

Пропущенные значения:
name      0
year      0
rating    0
time      0
dtype: int64


## Преобразуем числовые столбцы в правильные типы

In [8]:
df["year"] = pd.to_numeric(df["year"], errors='coerce')
df["rating"] = pd.to_numeric(df["rating"], errors='coerce')

## Удаляем строки с некорректными значениями (если есть)

In [9]:
df.dropna(subset=["year", "rating"], inplace=True)

## Выводим результат

In [10]:
print("\nОчищенный DataFrame:")
df.info()
df.head()


Очищенный DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    25 non-null     object 
 1   year    25 non-null     int64  
 2   rating  25 non-null     float64
 3   time    25 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 932.0+ bytes


,name,year,rating,time
0,The Shawshank Redemption,1994,9.3,2h 22m
1,The Godfather,1972,9.2,2h 55m
2,The Dark Knight,2008,9.1,2h 32m
3,The Godfather Part II,1974,9.0,3h 22m
4,12 Angry Men,1957,9.0,1h 36m
